# Part 1.4. Multivariable Linear Regression

In [0]:
import torch

## 1. Data Definition
`Quiz 1`, `Quiz 2`, `Quiz 3`에 따른 내 최종 점수를 예측해보자. 이 때 입력은 `Quiz 1`, `Quiz 2`, `Quiz 3` 3가지이며, 출력은 점수이다.

![4-1.png](./img/4-1.png)

In [0]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

## 2. Hypothesis
$H(x) = Wx + b$로 나타낼 수 있다. 이 때 $W$는 $\begin{bmatrix}w_1 & w_2 & w_3\end{bmatrix}$인 행렬이다. 

In [4]:
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

hypothesis = x_train.matmul(W) + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## 3. Training

In [6]:
# optimizer 정의
optimizer = torch.optim.SGD([W, b], lr=1e-5)

# 하이퍼 파라미터 설정
nb_epochs = 20

# 학습
for epoch in range(nb_epochs+1):
  hypothesis = x_train.matmul(W) + b
  cost = torch.mean((hypothesis - y_train) ** 2)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print("Epoch {:4d}/{} hypothesis: {} cost: {:.6f}".format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) cost: 287.936096
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) cost: 91.371071
Epoch    6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) cost: 29.758249
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) cost: 10.445267
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) cost: 4.391237
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) cost: 2.493121
Epoch   10/20 hypo

## 4. 더 간편하게!
`nn.Module`을 상속하여 모델을 생성하면 편하게 신경망을 생성할 수 있다.
* `nn.Linear(m, n)` : 선형 모델로 입력이 `m`, 출력이 `n`
* `forward()` : hypothesis를 계산
* `backward()` : 기울기를 계산

또한 `torch.nn.functional`에서는 여러가지 cost function을 제공한다. 자기가 원하는 cost function 함수를 호출하면 굳이 수식을 코드로 적지 않아도 된다.
* `F.mse_loss(pred, y)`, `F.l1_loss(pred, y)`, `F.smooth_l1_loss(pred, y)`

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

In [9]:
# 모델 정의 및 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

# 하이퍼 파라미터 설정
nb_epochs = 20

# 학습
for epoch in range(nb_epochs+1):
  hypothesis = model(x_train)
  cost = F.mse_loss(hypothesis, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print("Epoch {:4d}/{} hypothesis: {} cost: {:.6f}".format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch    0/20 hypothesis: tensor([-55.8703, -70.9245, -67.8984, -74.2329, -54.8300]) cost: 56385.769531
Epoch    1/20 hypothesis: tensor([36.8581, 40.5300, 41.9187, 45.3553, 30.1821]) cost: 17678.546875
Epoch    2/20 hypothesis: tensor([ 88.7730, 102.9296, 103.4011, 112.3082,  77.7777]) cost: 5545.882812
Epoch    3/20 hypothesis: tensor([117.8378, 137.8651, 137.8228, 149.7926, 104.4251]) cost: 1742.933838
Epoch    4/20 hypothesis: tensor([134.1096, 157.4245, 157.0940, 170.7786, 119.3444]) cost: 550.910034
Epoch    5/20 hypothesis: tensor([143.2192, 168.3754, 167.8832, 182.5278, 127.6976]) cost: 177.270966
Epoch    6/20 hypothesis: tensor([148.3189, 174.5067, 173.9235, 189.1057, 132.3746]) cost: 60.152580
Epoch    7/20 hypothesis: tensor([151.1736, 177.9397, 177.3051, 192.7882, 134.9935]) cost: 23.439976
Epoch    8/20 hypothesis: tensor([152.7714, 179.8620, 179.1983, 194.8499, 136.4601]) cost: 11.930044
Epoch    9/20 hypothesis: tensor([153.6656, 180.9385, 180.2580, 196.0040, 137.2816])

## 5. Minibatch Gradient Descent
데이터 개수가 10만개, 1억개 이렇게 많아지게 되면 한 번에 학습시키는 것이 버거울 수 있다. 계산도 느리고 그 계산량을 감당할 컴퓨터도 없다. 그래서 전체 데이터를 균일하게 나눠서 작은 단위로 한습시키는 **미니배치 학습**을 시킨다.   

![4-2.png](./img/4-2.png)

업데이트를 좀 더 빠르게 할 수 있다는 장점이 있지만 전체 데이터를 가지고 학습을 하지 않기에 잘못된 방향으로 업데이트가 될 수 있다.

![4-3.png](./img/4-3.png)


In [0]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]                
    self.y_data = [[152], [185], [180], [196], [142]]

  # 데이터셋의 총 데이터수
  def __len__(self):
    return len(self.x_data)

  # 어떤 idx를 받았을 때 그에 상응하는 입출력 데이터를 반환
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    
    return x, y

# 데이터셋 생성
dataset = CustomDataset()

In [0]:
from torch.utils.data import DataLoader

# batch_size는 미니배치의 크기로 보통 2의 제곱수로 설정
# shuffle은 각 epoch마다 데이터셋을 섞어서 학습 순서를 바꾸는 것
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

다음은 데이터를 3분할로 나누어 각각을 학습시키는 과정이다. `epoch` 20에, `batch_size` 3으로 따지자면 총 60번 학습을 하게 된다.

In [12]:
# 하이퍼 파라미터 설정
nb_epochs = 20

for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
 
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)

    # 매개변수 업데이트
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # len(dataloader)는 한 epoch당 미니 매치의 개수
    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()))

Epoch    0/20 Batch 1/3 Cost: 4.890475
Epoch    0/20 Batch 2/3 Cost: 7.519703
Epoch    0/20 Batch 3/3 Cost: 10.077447
Epoch    1/20 Batch 1/3 Cost: 2.296371
Epoch    1/20 Batch 2/3 Cost: 19.811586
Epoch    1/20 Batch 3/3 Cost: 4.386920
Epoch    2/20 Batch 1/3 Cost: 8.838188
Epoch    2/20 Batch 2/3 Cost: 7.477368
Epoch    2/20 Batch 3/3 Cost: 2.407289
Epoch    3/20 Batch 1/3 Cost: 6.112224
Epoch    3/20 Batch 2/3 Cost: 8.051439
Epoch    3/20 Batch 3/3 Cost: 8.984534
Epoch    4/20 Batch 1/3 Cost: 1.997134
Epoch    4/20 Batch 2/3 Cost: 8.750154
Epoch    4/20 Batch 3/3 Cost: 16.186945
Epoch    5/20 Batch 1/3 Cost: 10.016115
Epoch    5/20 Batch 2/3 Cost: 8.552842
Epoch    5/20 Batch 3/3 Cost: 6.859098
Epoch    6/20 Batch 1/3 Cost: 11.930544
Epoch    6/20 Batch 2/3 Cost: 11.542886
Epoch    6/20 Batch 3/3 Cost: 7.841043
Epoch    7/20 Batch 1/3 Cost: 8.123621
Epoch    7/20 Batch 2/3 Cost: 5.737539
Epoch    7/20 Batch 3/3 Cost: 12.446891
Epoch    8/20 Batch 1/3 Cost: 4.518116
Epoch    8/20 Batc